In [1]:
# Import necessary libraries and modules
from collections import deque  # Importing deque for efficient queue operations
from imutils.video import FileVideoStream  # Importing FileVideoStream for video streaming
import numpy as np  # Importing numpy for numerical operations
import cv2  # Importing OpenCV for computer vision tasks
import imutils  # Importing imutils for convenience functions
import time  # Importing time for time-related operations

# Define the lower and upper boundaries of the "ball color" in the HSV color space
ballColorLower = (20, 100, 100)  # Lower HSV values for ball color
ballColorUpper = (30, 255, 255)  # Upper HSV values for ball color
pts = deque(maxlen=64)  # Queue to store tracked points with a maximum length of 64
y_coords = []  # List to store y-coordinates of the detected object
dribble_counter = 0  # Counter to track dribbles

# Specify the path to the video file
video_path = "TEST.mp4"

# Open the video file
vs = FileVideoStream(video_path).start()  # Starting the file video stream
time.sleep(2.0)  # Allow the video file to warm up

# Create a popup window
cv2.namedWindow("Output", cv2.WINDOW_NORMAL)  # Creating a resizable named window

# Loop over the frames of the video
while True:
    # Grab the current frame
    frame = vs.read()  # Reading a frame from the video stream

    # If we reached the end of the video, break from the loop
    if frame is None:
        break

    # Preprocess the frame for ball detection
    frame = imutils.resize(frame, width=600)  # Resizing the frame to a width of 600 pixels
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)  # Applying Gaussian blur to the frame
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)  # Converting the frame to HSV color space

    # Create a mask for the specified ball color range and perform morphological operations
    mask = cv2.inRange(hsv, ballColorLower, ballColorUpper)  # Creating a mask for the specified color range
    mask = cv2.erode(mask, None, iterations=2)  # Eroding the mask to remove noise
    mask = cv2.dilate(mask, None, iterations=2)  # Dilating the mask to fill gaps in contours

    # Find contours in the mask
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # Finding contours in the mask
    cnts = imutils.grab_contours(cnts)  # Grabbing contours for compatibility
    center = None  # Initializing the center variable

    # Proceed if at least one contour was found
    if len(cnts) > 0:
        # Find the largest contour in the mask
        c = max(cnts, key=cv2.contourArea)  # Finding the largest contour
        ((x, y), radius) = cv2.minEnclosingCircle(c)  # Computing the minimum enclosing circle
        M = cv2.moments(c)  # Calculating moments of the contour
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))  # Calculating centroid coordinates

        # Proceed if the radius meets a minimum size
        if radius > 10:
            # Draw the circle and centroid on the frame and update the list of tracked points
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)  # Drawing the circle
            cv2.circle(frame, center, 5, (0, 0, 255), -1)  # Drawing the centroid
            y_coords.append(y)  # Adding y-coordinate to the list

    # Update the points queue
    pts.appendleft(center)  # Appending the centroid coordinates to the points deque

    # Loop over the set of tracked points to draw connecting lines
    for i in range(1, len(pts)):
        # Proceed if both tracked points are not None
        if pts[i - 1] is not None and pts[i] is not None:
            # Compute the thickness of the line and draw the connecting lines
            thickness = int(np.sqrt(64 / float(i + 1)) * 2.5)  # Computing thickness of the line
            cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)  # Drawing connecting lines between tracked points

    # Increment dribble_counter if conditions are met
    if len(y_coords) >= 7:
        # Check conditions for dribble detection
        if (y_coords[-1] < y_coords[-2] < y_coords[-3] < y_coords[-4]) and \
                (y_coords[-1] < y_coords[-6] < y_coords[-5] < y_coords[-4]):
            dribble_counter += 1

    # Display the frame in a popup window
    cv2.imshow("Output", frame)  # Showing the processed frame in a window titled "Output"
    key = cv2.waitKey(10) & 0xFF  # Waiting for a key press for 15 milliseconds
    # If the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break

# Stop the video stream
vs.stop()  # Stopping the video stream
# Close the popup window
cv2.destroyAllWindows()  # Closing all OpenCV windows

# Print the dribble counter
print("Dribble Count:", dribble_counter)


Dribble Count: 84
